#TP4

Paulo Freitas - A100053


Pedro Santos - A100110

### Enunciado

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
  - A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`.
    - o modo `Start` inicia o funcionamento com os valores iniciais das velocidades
    - no modo `Free`  não existe qualquer força de travagem;
    - no modo `Stopping` aplica-se a força de travagem alta;
    - no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa) com pequeno atrito ao solo;
    - no modo `Stopped` o veículo está imobilizado.
  - A componente contínua  do autómato usa variáveis contínuas $$\,V,v\,$$ para descrever a  `velocidade do corpo`   e a `velocidade linear das rodas`  ambas em relação so solo.
  - Assume-se que o sistema de travagem exerce uma força de atrito proporcional à diferença das duas velocidades.  A dinâmica contínua, as equações de fluxo, está descrita  abaixo.
  - Os “switchs” são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser  de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
  - É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

Faça

1. Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos.
2. A condição de segurança estabelece que o sistema não permaneça no modo `free`  ou no modo `blocked` mais do que $$\,\tau\,$$ segundos.
3. Defina um SFOTS que modele a discretização do autómato híbrido.
4. Verifique nesse modelo
    1. Que as condições de segurança são invariantes do sistema
    2. Que o sistema atinge o estado `stopped` eventualmente.



![](https://paper-attachments.dropboxusercontent.com/s_9896551CC5FAD2B2EB6E4EBC08522545FA66314D29FE6A5BE8E593259F8E8A37_1672830059198_ABS.png)

![](https://paper-attachments.dropboxusercontent.com/s_C5426E33053EA04D1E3446B871CC8485C43045FF7CAAA1FC86A9B3A8E09058F9_1574702123197_Documento.png)



  | Equações de Fluxo

  1. Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas. <br>
  
  2. No contacto rodas/solo a força de compressão é dada pelo o peso $P$ que é constante e independente do modo. Tem-se $f = a\, P$ sendo  $a$ a constante de atrito; o valor de $a$ depende do modo: é baixa em `Blocked` e alta nos restantes.<br>
  
  3. No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à diferença de velocidades<br><br>                                                                      $F =  c\, (V-v)$<br>      A  constante de proporcionalidade $c$ depende do modo: é elevada no modo `Stopping` e baixa nos outros.<br><br>
  
  4. As  equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`,<br>                                                        $(\dot{V} \,=\, -F)\,\land\, (\dot{v} \,=\, -a\, P  + F)$   <br>e , no modo `Blocked`,  a dinâmica do sistema é  regida por:<br>                                                         $(V = v) \;\land\; (\,\dot{v}\,=\, -a\, P\,)$<br>


  5. Tanto no modo `Blocked`  como no modo `Free`  existe um “timer” que impede que o controlo aí permaneça mais do que $\,\tau\,$segundos.  Os     $\mathsf{switch}(V,v,t,V',v',t')\,$  nesses modos devem forçar esta condição. <br>
  

  6. Todos os “switchs” devem ser construídos de  modo a impedir a existência de trajetórias de Zenão.<br>
  

  7. No instante inicial  o modo é  `Start` e  tem-se $\,V = v\,=\,V_0$  . A velocidade $V_0$ é  “input” do problema.



## ex uno

![Image of Yaktocat](https://i.ibb.co/p0vKbsW/image-1.png)

## Import das Bibliotecas

Para este exercício serão usadas as bibliotecas do pysmt e os sover z3 e msat.

In [ ]:
%%capture
!yes | pip install pysmt
!apt-get install libgmp3-dev
!yes | pysmt-install --z3

file = '/usr/local/lib/python3.10/dist-packages/pysmt/smtlib/parser/__init__.py'
with open(file, 'r') as f:
    code = f.read()
    new_code = code.replace('USE_CYTHON = True', 'USE_CYTHON = False')

with open(file, 'w') as f:
  f.write(new_code)

# Exercício 1.3

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT,BVType
from pysmt.logics import QF_NIA, QF_BV
import itertools
import matplotlib.pyplot as plt

In [2]:
START = 0
FREE = 1
STOPPING = 2
BLOCKED = 3
STOPPED = 4

In [3]:
vdif = 0.1
tmax = 0.1
a = 0.01
b = 0.5
c_stopping = 10
c = 0.5
P = 1000
velinicial = 20

In [4]:
def declare(i):
    s = {}

    s['t']      = Symbol('t' + str(i), REAL)                # TEMPO
    s['m']      = Symbol('m' + str(i), INT)                 # MODO
    s['vcorpo'] = Symbol('vcorpo' + str(i), REAL)           # VELOCIDADE CARRO
    s['vrodas'] = Symbol('vrodas' + str(i), REAL)           # VELOCIDADE RODAS
    s['timer']  = Symbol('timer' + str(i), REAL)            # TIMER

    return s

In [5]:
def init(state):
  T  = Equals(state['t'], Real(0))
  V1 = Equals(state['vcorpo'], Real(velinicial))
  V2 = Equals(state['vrodas'], Real(velinicial))
  M  = Equals(state['m'], Int(START))
  return And(T,V1,V2,M)


In [15]:
def trans(curr, prox):
    v_final = And(Equals(prox['vcorpo'], prox['vrodas']),
                  Equals(prox['vcorpo'], Real(0)),
                  Equals(prox['timer'], Real(0)))

  # untimed

    start_free = And(
        Equals(curr['m'], Int(START)),
        Equals(prox['m'], Int(FREE)),
        Equals(curr['t'], prox['t']),
        Equals(curr['vcorpo'], prox['vcorpo']),
        Equals(curr['vrodas'], prox['vrodas']),
        Equals(prox['timer'], Real(0)),
        Equals(curr['timer'], Real(0))
    )
    
    free_stopping = And(
        Equals(curr['m'], Int(FREE)),
        Equals(prox['m'], Int(STOPPING)),
        Equals(curr['t'], prox['t']),
        Equals(curr['vcorpo'], prox['vcorpo']),
        Equals(curr['vrodas'], prox['vrodas']),
        GE(curr['timer'], Real(tmax)),
        Equals(prox['timer'], Real(0))
    )
    
    stopping_blocked = And(
        Equals(curr['m'], Int(STOPPING)),
        Equals(prox['m'], Int(BLOCKED)),
        Equals(curr['t'], prox['t']),
        Equals(curr['vcorpo'], prox['vcorpo']),
        Equals(curr['vrodas'], prox['vrodas']),
        LE(Minus(curr['vcorpo'], curr['vrodas']), Real(vdif)),
        Equals(prox['timer'], Real(0))
    )
    
    blocked_free = And(
        Equals(curr['m'], Int(BLOCKED)),
        Equals(prox['m'], Int(FREE)),
        Equals(curr['t'], prox['t']),
        Equals(curr['vcorpo'], prox['vcorpo']),
        Equals(curr['vrodas'], prox['vrodas']),
        GE(curr['timer'], Real(tmax)),
        Equals(prox['timer'], Real(0))
    )
    
    free_stopped = And(
        Equals(curr['m'], Int(FREE)),
        Equals(prox['m'], Int(STOPPED)),  
        Equals(curr['t'], prox['t']),
        LE(curr['vcorpo'], Real(vdif)),
        LE(curr['vrodas'], Real(vdif)),
        v_final
    )
    
    stopping_stopped = And(
        Equals(curr['m'], Int(STOPPING)),
        Equals(prox['m'], Int(STOPPED)), 
        Equals(curr['t'], prox['t']),
        LE(curr['vcorpo'], Real(vdif)),
        LE(curr['vrodas'], Real(vdif)),
        v_final
    )
    
    blocked_stopped = And(
        Equals(curr['m'], Int(BLOCKED)),
        Equals(prox['m'], Int(STOPPED)),  
        Equals(curr['t'], prox['t']),
        LE(curr['vcorpo'], Real(vdif)),
        LE(curr['vrodas'], Real(vdif)),
        v_final
    )

    # timed
    intervalo = 0.1

    blocked_blocked = And(
        Equals(curr['m'], Int(BLOCKED)),
        Equals(prox['m'], Int(BLOCKED)),
        LT(curr['t'], prox['t']),
        Equals(Minus(prox['timer'], curr['timer']), Minus(prox['t'], curr['t'])),
        LE(prox['timer'], Real(tmax)),
        LE(curr['vcorpo'], Plus(curr['vrodas'], Real(vdif))),
        LE(prox['vcorpo'], Plus(prox['vrodas'], Real(vdif))),
        Equals(Minus(prox['vcorpo'], curr['vcorpo']), Times(Minus(Times(Real(-a), Real(P)), Real(b)), Real(intervalo))),
        Equals(Minus(prox['t'], curr['t']), Real(intervalo)),
        LE(curr['vrodas'], curr['vcorpo']),
        LE(prox['vrodas'], prox['vcorpo']),
        GE(prox['vcorpo'], Real(vdif)),
        GE(prox['vrodas'], Real(vdif))
    )

    free_free = And(
        Equals(curr['m'], Int(FREE)),
        Equals(prox['m'], Int(FREE)),
        LT(curr['t'], prox['t']),
        Equals(Minus(prox['timer'], curr['timer']), Minus(prox['t'], curr['t'])),
        LE(prox['timer'], Real(tmax)),
        Equals(Minus(prox['vcorpo'], curr['vcorpo']), Times(Minus(Times(Real(-c), Minus(curr['vcorpo'], curr['vrodas'])), Real(-b)), Real(intervalo))),
        Equals(Minus(prox['vrodas'], curr['vrodas']), Times(Plus(Times(Real(-a), Real(P)), Times(Real(c), Minus(curr['vcorpo'], curr['vrodas']))), Real(intervalo))),
        Equals(Minus(prox['t'], curr['t']), Real(intervalo)),
        LE(curr['vrodas'], curr['vcorpo']),
        LE(prox['vrodas'], prox['vcorpo']),
        GE(prox['vcorpo'], Real(vdif)),
        GE(prox['vrodas'], Real(vdif))
    )

                                                               
    
    stopping_stopping = And(
        Equals(curr['m'], Int(STOPPING)),
        Equals(prox['m'], Int(STOPPING)),
        Equals(curr['timer'], Real(0)),
        Equals(prox['timer'], Real(0)),
        Equals(Minus(prox['vcorpo'], curr['vcorpo']), Times(Minus(Times(Real(-c_stopping), Minus(curr['vcorpo'], curr['vrodas'])), Real(-b)), Real(intervalo))),
        Equals(Minus(prox['vrodas'], curr['vrodas']), Times(Plus(Times(Real(-a), Real(P)), Times(Real(c_stopping), Minus(curr['vcorpo'], curr['vrodas']))), Real(intervalo))), #linha que da erro
        Equals(Minus(prox['t'], curr['t']), Real(intervalo)),
        GT(Minus(curr['vcorpo'], curr['vrodas']), Real(vdif)),
        LE(curr['vrodas'], curr['vcorpo']),
        LE(prox['vrodas'], prox['vcorpo'])
    )
    
    stopped_stopped = And(
        Equals(curr['m'], Int(STOPPED)),
        Equals(prox['m'], Int(STOPPED)),
        Equals(prox['t'], curr['t']),
        Equals(prox['vrodas'], curr['vrodas']),
        Equals(prox['vcorpo'], curr['vcorpo']),
        Equals(curr['timer'], Real(0)),
        Equals(prox['timer'], Real(0))
    )
        
    return Or(start_free, free_stopping, stopping_blocked, blocked_free, free_stopped, stopping_stopped, blocked_stopped, free_free, stopping_stopping, blocked_blocked, stopped_stopped)

In [17]:
def gera_traco(declare, init, trans, k):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(k)]
        solver.add_assertion(init(states[0]))
        for i in range(k-1):
            solver.add_assertion(trans(states[i], states[i+1]))
        print('Solving')
        modes = ['START', 'FREE', 'STOPPING', 'BLOCKED', 'STOPPED']
        if solver.solve():
            for i in range(k):
                m = solver.get_value(states[i]["m"]).constant_value()
                print('t =', round(float(solver.get_value(states[i]["t"]).constant_value()),1),
                     '\nm =', modes[m],
                      '\nvcorpo =', round(float(solver.get_value(states[i]["vcorpo"]).constant_value()),1),
                      '\nvrodas =', round(float(solver.get_value(states[i]["vrodas"]).constant_value()),1),
                      '\ntimer =', float(solver.get_value(states[i]["timer"]).constant_value())
                     )
                print()
            print('w')
        else:
            print('L')
            
gera_traco(declare, init, trans, 10)

Solving
t = 0.0 
m = START 
vcorpo = 20.0 
vrodas = 20.0 
timer = 0.0

t = 0.0 
m = FREE 
vcorpo = 20.0 
vrodas = 20.0 
timer = 0.0

t = 0.1 
m = FREE 
vcorpo = 20.1 
vrodas = 19.0 
timer = 0.1

t = 0.1 
m = STOPPING 
vcorpo = 20.1 
vrodas = 19.0 
timer = 0.0

t = 0.2 
m = STOPPING 
vcorpo = 19.1 
vrodas = 18.9 
timer = 0.0

t = 0.2 
m = BLOCKED 
vcorpo = 19.1 
vrodas = 18.9 
timer = 0.0

t = 0.3 
m = BLOCKED 
vcorpo = 18.0 
vrodas = 18.0 
timer = 0.1

t = 0.3 
m = FREE 
vcorpo = 18.0 
vrodas = 18.0 
timer = 0.0

t = 0.4 
m = FREE 
vcorpo = 18.1 
vrodas = 17.0 
timer = 0.1

t = 0.4 
m = STOPPING 
vcorpo = 18.1 
vrodas = 17.0 
timer = 0.0

w


## Exercício 1.4

In [ ]:
def invariante(curr,prox):
  i1 = Implies(prox['t'] > curr['t'], prox['vcorpo'] < curr['vcorpo'])  # PERDE VEL AO LONGO DO TEMPO
  i2 = Implies(prox['t'] > curr['t'], prox['vrodas'] < curr['vrodas'])
  i3 = curr['vcorpo'] >= curr['vrodas']
  i4 = curr['timer']<= tmax

  return And(i1,i2,i3,i4)

In [ ]:
def bmc_always(declare,init,trans,inv,K):
  for k in range(2,K+1):
    s = Solver()
    traco = [declare(i) for i in range(k)]
    s.add(init(traco[0]))
    for i in range(k-1):
      s.add(trans(traco[i], traco[i+1]))

    s.add(Not(inv(traco[k-2], traco[k-1])))

    if s.check() == sat:
      m = s.model()
      for i in range(k):
        print(f"{k}Estado:", i)
        for v in traco[i]:
          res = m[traco[i][v]]
          if res.sort() != RealSort():
            print(v, '=', res)
          else:
            print(v, '=', float(res.numerator_as_long())/float(res.denominator_as_long()))
      return
  return "A propriedade é válida para traços de tamanho até " + str(K)

bmc_always(declare,init,trans,invariante, 70)

'A propriedade é válida para traços de tamanho até 70'

Queremos mostrar que o programa atingue, em algum momento o modo "STOPPED". Pela defenição do automato e, baseado no enunciado, chegamos rápidamente à conclusão que, o resultado é valido se no ultimo estado, o modo for "STOPPED", pois este modo é o estado terminal do automato.

Assim, basta provar que a ultima iteração de estados resulta em modo "STOPPED".

In [ ]:
def verificaStop(declare, init, trans, k):
  s = Solver()
  traco = [declare(i) for i in range(k)]
  s.add(init(traco[0]))
  for i in range(k-1):
    s.add(trans(traco[i], traco[i+1]))
  if s.check() == sat:
    m = s.model()
    res = m[traco[-1]['m']]
    if str(res) == 'STOPPED':
      print("A propriedade é valida, o modo do ultimo estado e:",res)

verificaStop(declare,init,trans,70)

A propriedade é valida, o modo do ultimo estado e: STOPPED
